In [1]:
import os
import pandas as pd
import numpy as np
import datetime
# import time
import pickle

data_dir = '../data'
processed_dir = '../processed'

headers = {
    'LSSC': [
        'No.',
        'Original air date',
        'Guest(s)',
        'Musical/entertainment guest(s)'
    ]
}

select = {
    'LSSC': headers['LSSC'],
}

# Concatening the Wikipedia scrapes into one csv file that will be read in.
def concatenate(prefix='LSSC'):
    files = os.listdir(data_dir)
    full = []
    for f in files:
        if not f.startswith(prefix):
            continue

        data = pd.read_csv(os.path.join(data_dir, f), engine='python', header=None, 
                           names=headers[prefix], index_col=False)
        data = data[select[prefix]]
        full.append(data)

    full = pd.concat(full, axis=0)

    full.to_csv(os.path.join(processed_dir, '{}.csv'.format(prefix)), header=select[prefix], index=False)

if __name__ == '__main__':
    concatenate('LSSC')

In [2]:
def read():
    wiki = pd.read_csv(os.path.join(processed_dir, 'LSSC.csv'), sep=',')
    return wiki

if __name__ == '__main__':
    wiki = read()

# EXPLORATORY DATA ANALYSIS

In [3]:
# Will need to scan the expanded table for errors and inconsistencies.
pd.set_option('display.max_rows', 2200)

In [4]:
# Dropping rows with null values in the Guests column.
wiki = wiki.dropna(axis=0, subset=['Guest(s)'])

# Dropping rows with 'Guest(s)' in the Guests column.
wiki = wiki[wiki['Guest(s)'] != 'Guest(s)']

wiki = wiki.reset_index()

# Setting cutoff point as the March 22nd, 2019, episode. This was when I first pulled the data.
wiki = wiki[:719]

# Dropping 'index' column. Not needed.
wiki = wiki.drop(['index'], axis=1)

## Splitting guests into multiple columns

In [5]:
wiki[['Guest_1','Guest_2','Guest_3','Guest_4', 'Guest_5']] = wiki['Guest(s)'].str.split(',', expand=True)
wiki[['Guest_6','Guest_7','Guest_8']] = wiki['Musical/entertainment guest(s)'].str.split(',', expand=True)

In [6]:
# Converting air date to datetime format and create new column for it.
wiki['date'] = pd.to_datetime(wiki['Original air date'])

# Sorting by air date in ascending order.
wiki = wiki.sort_values(by='date')
wiki = wiki.reset_index()
wiki = wiki.drop(['index','Original air date'], axis=1)

In [7]:
wiki = wiki.apply(lambda x : x.str.strip() if x.dtype=='object' else x)

## Main guest cleaning
In a typical episode of the show, there are three guests. The first two (typically occupying `Guest_1` and `Guest_2` columns, respectively) conduct interviews with Colbert. The final guest is a musician or some other form of entertainer (occupying the `Guest_6` column and the others if needed.)

Checking the `Guest_3` column lets us see if there are interviews that consist of multiple guests simultaneously.

In [8]:
wiki.Guest_3.value_counts()

Brian Greene                             3
Isaac Mizrahi                            2
Gilbert Gottfried                        2
Lewis Black                              2
Roy Wood Jr.                             2
Triumph                                  2
Judy Gold                                2
Tommy Vietor                             2
Ken Burns                                2
Paula Poundstone                         2
Louie Anderson                           2
Megyn Kelly                              2
Jon Lovett                               2
Charlamagne Tha God                      2
Michaela Watkins                         2
George Saunders                          1
Michael Eric Dyson                       1
Peter Serafinowicz                       1
Paul Mecurio                             1
George Takei                             1
Florent Groberg                          1
Jonah Reider                             1
Mike Epps                                1
Martha Stew

In [9]:
wiki.loc[(wiki.date=='2016-02-07')] = wiki.loc[
    (wiki.date=='2016-02-07')].replace({'Guest_4' : None}, 'Keegan-Michael Key, Jordan Peele')

wiki.loc[(wiki.date=='2016-02-07')] = wiki.loc[
    (wiki.date=='2016-02-07')].replace({'Guest_6' : 'Keegan-Michael Key & Jordan Peele'}, 'None')

In [10]:
wiki.loc[(wiki.date=='2016-02-18')] = wiki.loc[
    (wiki.date=='2016-02-18')].replace({'Guest_3' : 'Triumph'}, 'Triumph the Insult Comic Dog')

wiki.loc[(wiki.date=='2016-02-18')] = wiki.loc[
    (wiki.date=='2016-02-18')].replace({'Guest_4' : 'the Insult Comic Dog'}, 'None')

In [11]:
wiki.loc[(wiki.date=='2016-11-09')] = wiki.loc[
    (wiki.date=='2016-11-09')].replace({'Guest_3' : 'Triumph'}, 'Triumph the Insult Comic Dog')

wiki.loc[(wiki.date=='2016-11-09')] = wiki.loc[
    (wiki.date=='2016-11-09')].replace({'Guest_4' : 'the Insult Comic Dog'}, 'None')

In [12]:
wiki.loc[(wiki.date=='2017-04-05')] = wiki.loc[
    (wiki.date=='2017-04-05')].replace({'Guest_2' : 'Jon Favreau'}, 'Pod Save America')

wiki.loc[(wiki.date=='2017-04-05')] = wiki.loc[
    (wiki.date=='2017-04-05')].replace({'Guest_3' : 'Jon Lovett'}, 'None')

wiki.loc[(wiki.date=='2017-04-05')] = wiki.loc[
    (wiki.date=='2017-04-05')].replace({'Guest_4' : 'Tommy Vietor'}, 'None')

In [13]:
wiki.loc[(wiki.date=='2018-08-16')] = wiki.loc[
    (wiki.date=='2018-08-16')].replace({'Guest_1' : 'Tommy Vietor'}, 'Pod Save America')

wiki.loc[(wiki.date=='2018-08-16')] = wiki.loc[
    (wiki.date=='2018-08-16')].replace({'Guest_2' : 'Jon Favreau'}, 'Constance Wu')

wiki.loc[(wiki.date=='2018-08-16')] = wiki.loc[
    (wiki.date=='2018-08-16')].replace({'Guest_3' : 'Jon Lovett'}, 'None')

wiki.loc[(wiki.date=='2018-08-16')] = wiki.loc[
    (wiki.date=='2018-08-16')].replace({'Guest_3' : 'Constance Wu'}, 'None')

In [14]:
wiki.loc[(wiki.date=='2016-04-28')] = wiki.loc[
    (wiki.date=='2016-04-28')].replace({'Guest_1' : 'Julianna Margulies'}, 'Julianna Margulies, Christine Baranski, Matt Czuchry')

wiki.loc[(wiki.date=='2016-04-28')] = wiki.loc[
    (wiki.date=='2016-04-28')].replace({'Guest_2' : 'Christine Baranski & Matt Czuchry; Hank Azaria'}, 'Hank Azaria')

In [15]:
wiki.loc[(wiki.date=='2018-01-30')] = wiki.loc[
    (wiki.date=='2018-01-30')].replace({'Guest_1' : 'Jon Favreau'}, 'Pod Save America')

wiki.loc[(wiki.date=='2018-01-30')] = wiki.loc[
    (wiki.date=='2018-01-30')].replace({'Guest_2' : 'Jon Lovett'}, '2 Dope Queens')

wiki.loc[(wiki.date=='2018-01-30')] = wiki.loc[
    (wiki.date=='2018-01-30')].replace({'Guest_3' : 'Tommy Vietor'}, 'None')

wiki.loc[(wiki.date=='2018-01-30')] = wiki.loc[
    (wiki.date=='2018-01-30')].replace({'Guest_4' : 'Jessica Williams'}, 'None')

wiki.loc[(wiki.date=='2018-01-30')] = wiki.loc[
    (wiki.date=='2018-01-30')].replace({'Guest_5' : 'Phoebe Robinson'}, 'None')

In [16]:
wiki.loc[(wiki.date=='2018-10-23')] = wiki.loc[
    (wiki.date=='2018-10-23')].replace({'Guest_1' : 'Jon Favreau'}, 'Pod Save America')

wiki.loc[(wiki.date=='2018-10-23')] = wiki.loc[
    (wiki.date=='2018-10-23')].replace({'Guest_2' : 'Jon Lovett'}, 'Charlamagne tha God')

wiki.loc[(wiki.date=='2018-10-23')] = wiki.loc[
    (wiki.date=='2018-10-23')].replace({'Guest_3' : 'Tommy Vietor'}, 'None')

wiki.loc[(wiki.date=='2018-10-23')] = wiki.loc[
    (wiki.date=='2018-10-23')].replace({'Guest_4' : 'Charlamagne tha God'}, 'None')

In [17]:
wiki.loc[(wiki.date=='2017-03-29')] = wiki.loc[
    (wiki.date=='2017-03-29')].replace({'Guest_3' : 'Luke Bryan & Dierks Bentley'}, 'Luke Bryan, Dierks Bentley')

wiki.loc[(wiki.date=='2017-03-29')] = wiki.loc[
    (wiki.date=='2017-03-29')].replace({'Guest_6' : 'Luke Bryan & Dierks Bentley'}, 'None')

In [18]:
wiki.loc[(wiki.date=='2015-12-16')] = wiki.loc[
    (wiki.date=='2015-12-16')].replace({'Guest_6' : 'Henry Rollins'}, 'None')

In [19]:
wiki.loc[(wiki.date=='2017-04-26')] = wiki.loc[
    (wiki.date=='2017-04-26')].replace({'Guest_3' : 'Dave & Virginia Grohl'}, 'Dave Grohl')

In [20]:
wiki.loc[(wiki.date=='2016-07-15')] = wiki.loc[
    (wiki.date=='2016-07-15')].replace({'Guest_2' : 'Gayle King & Norah O\'Donnell'}, 'Gayle King, Norah O\'Donnell')

In [21]:
wiki.loc[(wiki.date=='2016-11-07')] = wiki.loc[
    (wiki.date=='2016-11-07')].replace({'Guest_6' : 'Stevie Wonder'}, 'None')

In [22]:
wiki.loc[(wiki.date=='2016-01-12')] = wiki.loc[
    (wiki.date=='2016-01-12')].replace({'Guest_3' : 'Laura Ricciardi & Moira Demos'}, 'Laura Ricciardi, Moira Demos')

In [23]:
wiki.loc[(wiki.date=='2016-07-28')] = wiki.loc[
    (wiki.date=='2016-07-28')].replace({'Guest_1' : 'Abbi Jacobson & Ilana Glazer'}, 'Abbi Jacobson, Ilana Glazer')

In [24]:
wiki.loc[(wiki.date=='2018-12-13')] = wiki.loc[
    (wiki.date=='2018-12-13')].replace({'Guest_2' : 'Joaquín & Julian Castro'}, 'Joaquín Castro, Julian Castro')

In [25]:
wiki.loc[(wiki.date=='2018-06-28')] = wiki.loc[
    (wiki.date=='2018-06-28')].replace({'Guest_2' : 'Eric Andre & Derrick Beckles'}, 'Eric Andre, Derrick Beckles')

In [26]:
wiki.loc[(wiki.date=='2016-04-21')] = wiki.loc[
    (wiki.date=='2016-04-21')].replace({'Guest_2' : 'Leslie Odom'}, 'Leslie Odom Jr.')

wiki.loc[(wiki.date=='2016-04-21')] = wiki.loc[
    (wiki.date=='2016-04-21')].replace({'Guest_3' : 'Jr.'}, 'None')

In [27]:
wiki.loc[(wiki.date=='2015-10-16')] = wiki.loc[
    (wiki.date=='2015-10-16')].replace({'Guest_2' : 'Guillermo del Toro'},
                                       'Guillermo del Toro, Jessica Chastain, Tom Hiddleston, Mia Wasikowska')

wiki.loc[(wiki.date=='2015-10-16')] = wiki.loc[
    (wiki.date=='2015-10-16')].replace({'Guest_3' : 'Jessica Chastain'}, 'None')

wiki.loc[(wiki.date=='2015-10-16')] = wiki.loc[
    (wiki.date=='2015-10-16')].replace({'Guest_4' : 'Tom Hiddleston'}, 'None')

wiki.loc[(wiki.date=='2015-10-16')] = wiki.loc[
    (wiki.date=='2015-10-16')].replace({'Guest_5' : 'Mia Wasikowska'}, 'None')

In [28]:
wiki.loc[(wiki.date=='2015-09-18')] = wiki.loc[
    (wiki.date=='2015-09-18')].replace({'Guest_6' : 'Robert Fairchild and Leanne Cope'}, 'Robert Fairchild, Leanne Cope')

wiki.loc[(wiki.date=='2015-09-18')] = wiki.loc[
    (wiki.date=='2015-09-18')].replace({'Guest_2' : 'Senator Bernie Sanders'}, 'Bernie Sanders')

In [29]:
wiki.loc[(wiki.date=='2019-02-05')] = wiki.loc[
    (wiki.date=='2019-02-05')].replace({'Guest_1' : 'Norah O\'Donnell'}, 'Norah O\'Donnell, John Dickerson, Gayle King, Bianna Golodryga')

wiki.loc[(wiki.date=='2019-02-05')] = wiki.loc[
    (wiki.date=='2019-02-05')].replace({'Guest_2' : 'John Dickerson'}, 'Spike Lee')

wiki.loc[(wiki.date=='2019-02-05')] = wiki.loc[
    (wiki.date=='2019-02-05')].replace({'Guest_3' : 'Gayle King'}, 'None')

wiki.loc[(wiki.date=='2019-02-05')] = wiki.loc[
    (wiki.date=='2019-02-05')].replace({'Guest_4' : 'Bianna Golodryga'}, 'None')

wiki.loc[(wiki.date=='2019-02-05')] = wiki.loc[
    (wiki.date=='2019-02-05')].replace({'Guest_5' : 'Spike Lee'}, 'None')

In [30]:
wiki.loc[(wiki.date=='2017-02-13')] = wiki.loc[
    (wiki.date=='2017-02-13')].replace({'Guest_3' : 'Rick & Marty Lagina'}, 'Rick Lagina, Marty Lagina')

In [31]:
wiki.loc[(wiki.date=='2017-05-09')] = wiki.loc[
    (wiki.date=='2017-05-09')].replace({'Guest_2' : 'Samantha Bee'}, 'Samantha Bee, John Oliver, Ed Helms, Rob Corddry')

wiki.loc[(wiki.date=='2017-05-09')] = wiki.loc[
    (wiki.date=='2017-05-09')].replace({'Guest_3' : 'John Oliver'}, 'None')

wiki.loc[(wiki.date=='2017-05-09')] = wiki.loc[
    (wiki.date=='2017-05-09')].replace({'Guest_4' : 'Ed Helms'}, 'None')

wiki.loc[(wiki.date=='2017-05-09')] = wiki.loc[
    (wiki.date=='2017-05-09')].replace({'Guest_5' : 'Rob Corddry'}, 'None')

In [32]:
wiki.loc[(wiki.date=='2015-12-09')] = wiki.loc[
    (wiki.date=='2015-12-09')].replace({'Guest_1' : 'Michelle Dockery'}, 'Michelle Dockery, Hugh Bonneville, Allen Leech')

wiki.loc[(wiki.date=='2015-12-09')] = wiki.loc[
    (wiki.date=='2015-12-09')].replace({'Guest_2' : 'Hugh Bonneville'}, 'None')

wiki.loc[(wiki.date=='2015-12-09')] = wiki.loc[
    (wiki.date=='2015-12-09')].replace({'Guest_3' : 'Allen Leech'}, 'None')

In [33]:
wiki.loc[(wiki.date=='2015-10-16')] = wiki.loc[
    (wiki.date=='2015-10-16')].replace({'Guest_2' : 'T. J. Miller'}, 'TJ Miller')

In [34]:
wiki.loc[(wiki.date=='2016-03-02')] = wiki.loc[
    (wiki.date=='2015-03-02')].replace({'Guest_2' : 'John Stamos'}, 'John Stamos, Bob Saget, Dave Coulier')

wiki.loc[(wiki.date=='2016-03-02')] = wiki.loc[
    (wiki.date=='2015-03-02')].replace({'Guest_3' : 'Bob Saget'}, 'None')

wiki.loc[(wiki.date=='2016-03-02')] = wiki.loc[
    (wiki.date=='2015-03-02')].replace({'Guest_4' : 'Dave Coulier'}, 'None')

In [35]:
wiki.loc[(wiki.date=='2015-11-24')] = wiki.loc[
    (wiki.date=='2015-11-24')].replace({'Guest_6' : 'Carly Simon'}, 'None')

In [36]:
wiki.loc[(wiki.date=='2015-11-10')] = wiki.loc[
    (wiki.date=='2015-11-10')].replace({'Guest_3' : 'Bruce Campbell & Lucy Lawless'}, 'Bruce Campbell, Lucy Lawless')

In [37]:
wiki.loc[(wiki.date=='2015-09-30')] = wiki.loc[
    (wiki.date=='2015-09-30')].replace({'Guest_3' : 'Bill Withers & Ed Sheeran'}, 'Bill Withers, Ed Sheeran')

In [38]:
wiki.loc[(wiki.date=='2016-02-03')] = wiki.loc[
    (wiki.date=='2016-02-03')].replace({'Guest_2' : 'Mark & Jay Duplass'}, 'Mark Duplass')

wiki.loc[(wiki.date=='2016-02-03')] = wiki.loc[
    (wiki.date=='2016-02-03')].replace({'Guest_6' : 'Anderson Paak and The Free Nationals'}, 'Anderson Paak, The Free Nationals')

In [39]:
wiki.loc[(wiki.date=='2016-06-09')] = wiki.loc[
    (wiki.date=='2016-06-09')].replace({'Guest_3' : 'Governors Gary Johnson & William Weld'}, 'Gary Johnson, William Weld')

In [40]:
wiki.loc[(wiki.date=='2016-10-31')] = wiki.loc[
    (wiki.date=='2016-10-31')].replace({'Guest_2' : 'J. B. Smoove'}, 'JB Smoove')

wiki.loc[(wiki.date=='2016-10-31')] = wiki.loc[
    (wiki.date=='2016-10-31')].replace({'Guest_3' : 'The Ghost Brothers'}, 'Ghost Brothers')

In [41]:
wiki.loc[(wiki.date=='2019-01-30')] = wiki.loc[
    (wiki.date=='2019-01-30')].replace({'Guest_1' : 'John Heilemann'}, 'John Heilemann, Mark McKinnon, Alex Wagner')

wiki.loc[(wiki.date=='2019-01-30')] = wiki.loc[
    (wiki.date=='2019-01-30')].replace({'Guest_2' : 'Mark McKinnon'}, 'None')

wiki.loc[(wiki.date=='2019-01-30')] = wiki.loc[
    (wiki.date=='2019-01-30')].replace({'Guest_3' : 'Alex Wagner'}, 'None')

In [42]:
wiki.loc[(wiki.date=='2016-10-04')] = wiki.loc[
    (wiki.date=='2016-10-04')].replace({'Guest_3' : 'Paul F. Tompkins'}, 'Paul Tompkins')

In [43]:
wiki.loc[(wiki.date=='2016-02-15')] = wiki.loc[
    (wiki.date=='2016-02-15')].replace({'Guest_2' : 'Hailey Clauson & Ashley Graham'}, 'Hailey Clauson, Ashley Graham')

In [44]:
wiki.loc[(wiki.date=='2019-02-04')] = wiki.loc[
    (wiki.date=='2019-02-04')].replace({'Guest_1' : 'Taraji P. Henson'}, 'Taraji Henson')

In [45]:
wiki.loc[(wiki.date=='2017-03-07')] = wiki.loc[
    (wiki.date=='2017-03-07')].replace({'Guest_3' : 'General Michael Hayden'}, 'Michael Hayden')

In [46]:
# Tracking the values in the `Guest_3` feature to see if their corresponding rows are already good to go or in need of
# modification.
OK = ['Brian Grene', 'Michaela Watkins', 'Paula Poundstone',  'Judy Gold', 'Gilbert Gottfried', 'Lewis Black',
     'Ken Burns', 'Charlamagne Tha God', 'Roy Wood Jr.', 'Louie Anderson', 'George Takei', 'Chris Wallace', 'Robin Thede', 
      'Ta-Nehisi Coates', 'John McWhorter', 'Chris Gethard', 'Peter Serafinowicz', 'Marina Franklin', 'Jerrod Carmichael',
      'John Irving', 'Amy Cuddy', 'Steve Kornacki', 'Morgan Spurlock', 'Michael Showalter', 'Michelle Wolf', 
      'Thomas L. Friedman', 'Max Brooks', 'Buzz Aldrin', 'Robert Klein', 'Jeff Watson', 'Tom Segura', 'Adam Richman',
      'April Ryan', 'Jordan Klepper', 'Tom Papa', 'Jen Kirkman', 'RuPaul Charles', 'Jena Friedman', 'Christian Borle',
      'Jon Glaser', 'Andrew Sullivan', 'Olivia Hallisey', 'Rob Huebel', 'Michael Pollan', 'Wyatt Cenac', 'Yvette Nicole Brown',
      'Nell Scovell', 'Ben Sinclair', 'John Waters', 'Adam Savage', 'PewDiePie', 'Scott Rogowsky', 'Ronny Chieng',
      'Michael Lewis', 'Carly Fleischmann', 'Paul Mecurio', 'H. Jon Benjamin', 'Dr. Eugenia Cheng', 'Moshe Kasher', 
      'Chesley Sullenberger', 'Niecy Nash', 'Malcolm Gladwell', 'George Church', 'Joe Walsh', 'Florent Groberg', 
      'Sam Richardson', 'Mike Epps', 'Asa Butterfield', 'Robert Smigel', 'Andy Daly', 'David Ortiz', 'DeRay Mckesson',
      'Brian Huskey', 'Sonequa Martin-Green', 'Daniel Boulud', 'Jackson Galaxy', 'June Diane Raphael', 'Tony Rock',
      'Alton Brown', 'John Avlon', '2 Chainz', 'Sarah Koenig', 'Ron Suskind', 'Jonah Reider', 'Dan Levy', 'Max Greenfield',
      'David Chang', 'Senator Elizabeth Warren', 'Spike Feresten', 'Erin Brockovich', 'Andrew Delbanco', 'Jack Maxwell',
      'George Saunders', 'Jorge Ramos', 'Derek DelGaudio', 'Dean Baquet', 'Nate Silver', 'Sean Evans', 'W. Kamau Bell',
      'Carrie Brownstein', 'Congressman John Lewis', 'Mary Elizabeth Winstead', 'Bret Baier', 'Paul Scheer', 'Sarah McDaniel',
      'Martha Stewart', 'Ezra Edelman', 'Senator Amy Klobuchar', 
     ]

cleaned = ['Megyn Kelly', 'Triumph', 'Jon Lovett', 'Isaac Mizrahi', 'Luke Bryan & Dierks Bentley', 'Henry Rollins', 
          'Dave & Virginia Grohl', 'DeRay McKesson', 'Katherine Waterston', 'Laura Ricciardi & Moira Demos', 'Eric André',
           'Ira Madison III', 'Alexandria Ocasio-Cortez', 'Jr.', 'Jessica Chastain', 'Christopher Wheeldon', 'Gayle King',
           'Rick & Marty Lagina', 'John Oliver', 'Allen Leech', 'Oliver Stone', 'Bob Saget', 'Carly Simon', 
           'Bruce Campbell & Lucy Lawless', 'Bill Withers & Ed Sheeran', 'Michael Eric Dyson', 
           'Governors Gary Johnson & William Weld', 'John Heilemann', 'Marie Kondo', 
          ]

In [47]:
wiki = wiki.fillna('None')

# 10-10-2016 rerun of 8-22-2016 episode
wiki = wiki[wiki['No.']!='-']
wiki = wiki[wiki['No.']!='None']

# 4-6-2017 rerun of 3-15-2017 episode
wiki = wiki[wiki['No.']!='324']

wiki = wiki.sort_values(by='date')
wiki = wiki.reset_index()
wiki = wiki.drop(['index'], axis=1)

wiki['date'] = pd.to_datetime(wiki['date'], format='%Y-%m-%d')

## Music and entertainment guests
As we did with the "regular" guests, we can search for commas within the `Musical/entertainment guest(s)` column to see if there were multiple guests of this type.

In [48]:
wiki[wiki['Musical/entertainment guest(s)'].str.contains(',')]

,No.,Guest(s),Musical/entertainment guest(s),Guest_1,Guest_2,Guest_3,Guest_4,Guest_5,Guest_6,Guest_7,Guest_8,date
115,117,"Sarah Paulson, Tatiana Maslany","Wynton Marsalis, Lil Buck & Jared Grimes",Sarah Paulson,Tatiana Maslany,None,None,None,Wynton Marsalis,Lil Buck & Jared Grimes,None,2016-03-31
153,155,"Laura Linney, John Leguizamo","Gary Clark, Jr.",Laura Linney,John Leguizamo,None,None,None,Gary Clark,Jr.,None,2016-06-08
376,379,"Al Gore, Issa Rae","Sufjan Stevens, Nico Muhly, Bryce Dessner & Ja...",Al Gore,Issa Rae,None,None,None,Sufjan Stevens,Nico Muhly,Bryce Dessner & James McAlister,2017-07-17
381,384,"Paul Bettany, Laura Benanti","Tyler, The Creator",Paul Bettany,Laura Benanti,None,None,None,Tyler,The Creator,None,2017-07-24
542,544,Jake Tapper,"Erik Bergstrom, Liam Payne & J Balvin",Jake Tapper,None,None,None,None,Erik Bergstrom,Liam Payne & J Balvin,None,2018-05-17
567,569,"Gordon Ramsay, Andrew Rannells","Shye Ben Tzur, Jonny Greenwood & The Rajasthan...",Gordon Ramsay,Andrew Rannells,None,None,None,Shye Ben Tzur,Jonny Greenwood & The Rajasthan Express,None,2018-07-09
598,600,Sharon Osbourne,"Kathleen Madigan, Jay Rock",Sharon Osbourne,None,None,None,None,Kathleen Madigan,Jay Rock,None,2018-09-07
695,697,Jeff Goldblum,"Jacques Torres, The Marcus King Band",Jeff Goldblum,None,None,None,None,Jacques Torres,The Marcus King Band,None,2019-02-15


In [49]:
wiki.loc[(wiki.date=='2016-03-31')] = wiki.loc[
    (wiki.date=='2016-03-31')].replace({'Guest_6' : 'Wynton Marsalis'}, 'Wynton Marsalis, Lil Buck, Jared Grimes')

wiki.loc[(wiki.date=='2016-03-31')] = wiki.loc[
    (wiki.date=='2016-03-31')].replace({'Guest_7' : 'Lil Buck & Jared Grimes'}, 'None')

In [50]:
wiki.loc[(wiki.date=='2016-06-08')] = wiki.loc[
    (wiki.date=='2016-06-08')].replace({'Guest_6' : 'Gary Clark'}, 'Gary Clark Jr.')

wiki.loc[(wiki.date=='2016-06-08')] = wiki.loc[
    (wiki.date=='2016-06-08')].replace({'Guest_7' : 'None'}, 'None')

In [51]:
wiki.loc[(wiki.date=='2017-07-17')] = wiki.loc[
    (wiki.date=='2017-07-17')].replace({'Guest_6' : 'Sufjan Stevens'}, 'Sufjan Stevens, Nico Muhly, Bryce Dessner, James McAlister')

wiki.loc[(wiki.date=='2017-07-17')] = wiki.loc[
    (wiki.date=='2017-07-17')].replace({'Guest_7' : 'Nico Muhly'}, 'None')

wiki.loc[(wiki.date=='2017-07-17')] = wiki.loc[
    (wiki.date=='2017-07-17')].replace({'Guest_8' : 'Bryce Dessner & James McAlister'}, 'None')

In [52]:
wiki.loc[(wiki.date=='2017-07-24')] = wiki.loc[
    (wiki.date=='2017-07-24')].replace({'Guest_6' : 'Tyler'}, 'Tyler the Creator')

wiki.loc[(wiki.date=='2017-07-24')] = wiki.loc[
    (wiki.date=='2017-07-24')].replace({'Guest_7' : 'The Creator'}, 'None')

In [53]:
wiki.loc[(wiki.date=='2018-05-17')] = wiki.loc[
    (wiki.date=='2018-05-17')].replace({'Guest_7' : 'Liam Payne & J Balvin'}, 'Liam Payne, J. Balvin')

In [54]:
wiki.loc[(wiki.date=='2018-05-17')] = wiki.loc[
    (wiki.date=='2018-05-17')].replace({'Guest_6' : 'Shye Ben Tzur'}, 'Shye Ben Tzur, Jonny Greenwood, The Rajasthan Express')

wiki.loc[(wiki.date=='2018-05-17')] = wiki.loc[
    (wiki.date=='2018-05-17')].replace({'Guest_7' : 'Jonny Greenwood & The Rajasthan Express'}, 'None')

In [55]:
wiki.loc[(wiki.date=='2019-02-15')] = wiki.loc[
    (wiki.date=='2019-02-15')].replace({'Guest_7' : 'The Marcus King Band'}, 'Marcus King Band')

In [56]:
wiki_new = pd.melt(wiki, id_vars=['date'], value_vars=['Guest_1','Guest_2','Guest_3','Guest_4','Guest_5',
                                                       'Guest_6','Guest_7','Guest_8'], value_name='Guest')
wiki_new = wiki_new.sort_values(by=['date','variable'])
wiki_new = wiki_new[(wiki_new.Guest != None) & (wiki_new.Guest != 'None')]
wiki_new = wiki_new.reset_index()
wiki_new = wiki_new.drop(['index'], axis=1)

# YOUTUBE VIDEOS
Now it's time to bring in the YouTube pull. I tried multiple API scrapers, but none of them could properly handle the infinite scroll of a YouTube channel's videos page. All of them stopped at the bottom of the "first page" without initiating another page-down to capture more videos' data.

I thus had to resort to using this tool (https://tools.digitalmethods.net/netvizz/youtube/mod_channel_info.php) created by the University of Amsterdam's Digital Methods Initiative. It saves the pull as a TAB file, so it must be converted to a CSV before it can be read in.

In [57]:
youtube = pd.read_csv('../LSSC_YT_Videos.csv', low_memory=False)

In [58]:
youtube['date'] = pd.to_datetime(youtube['publishedAtSQL'])
youtube['date'] = youtube['date'].dt.date
youtube = youtube.drop(['channelId','channelTitle','publishedAt','videoCategoryId','videoCategoryLabel','duration',
                        'dimension','caption','definition','licensedContent','favoriteCount','publishedAtSQL'], axis=1)

# YouTube uploads of 'Late Show' videos in the early hours of the day after its corresponding episode airs on linear TV.
# Accounting for this delay by subracting one day off the upload date.
youtube['date'] = youtube['date'] - datetime.timedelta(days=1)
youtube['date'] = pd.to_datetime(youtube['date'])

# MERGING DATAFRAMES

In [59]:
data = pd.merge(wiki_new, youtube, on='date')
data.head()

,date,variable,Guest,position,videoId,videoTitle,videoDescription,durationSec,viewCount,likeCount,dislikeCount,commentCount
0,2015-09-08,Guest_1,George Clooney,5153,cSB7SRAReY0,George Clooney Extended Interview,George talks about his work on behalf of Darfu...,259,360057,2286,248,149
1,2015-09-08,Guest_1,George Clooney,5154,qmGflkMYi5k,Jeb Gets His Trump On,Stephen helps Jeb prep for the next GOP debate...,122,232577,2164,333,263
2,2015-09-08,Guest_1,George Clooney,5155,XG78zEhuPCI,Introducing The New Ed Sullivan Theater,Stephen shows off his completely renovated set...,344,430547,3137,460,271
3,2015-09-08,Guest_1,George Clooney,5156,F5zjVUZA7rY,All You Can Trump Buffet,Stephen knows talking about Donald Trump is te...,342,4590314,36667,2504,3649
4,2015-09-08,Guest_1,George Clooney,5157,lVlfN_H3Lzk,Jeb Bush Has Something Nice To Say About Obama,Jeb Bush goes rogue and says something not tot...,92,588030,2768,458,526


In [60]:
# Removing titles that will interfere with the Google Knowledge API that we'll eventually run.
titles = ['Secretary of State ', 'Secretary General ', 'Senator ', 'Vice President ', 'Dr. ', 'Representative ', 'Congressman ',
          'Governor ', 'Governors ', 'The ', 'the ','Justice ']

for title in titles:
    data['Guest'] = data['Guest'].str.replace(title, '')

In [61]:
# While a single guest slot can contain multiple guests, we're extracting the first guest and using this as the unique
# identifier for each row.
data['guest_first'] = data.Guest.str.split().str.slice(stop=2)
data['guest_first'] = data.guest_first.apply(' '.join)

In [62]:
data = data[ data.apply(lambda row: row.guest_first in row.videoTitle, axis=1) ]
data = data.reset_index()
data = data.drop(['index'], axis=1)

In [63]:
# Using the first two words as the first name unique identifier works for most entries, but not all. Cleaning these up.
data['guest_1'] = data.Guest.str.split().str[:2].str.join(' ')
data.loc[data.guest_1 == 'Michael C.', 'guest_1'] = 'Michael C. Hall'
data.loc[data.guest_1 == 'Samuel L.', 'guest_1'] = 'Samuel L. Jackson'
data.loc[data.guest_1 == 'J. J.', 'guest_1'] = 'JJ Abrams'
data.loc[data.guest_1 == 'William H.', 'guest_1'] = 'William H. Macy'
data.loc[data.guest_1 == 'J. K.', 'guest_1'] = 'JK Simmons'
data.loc[data.guest_1 == 'W. Kamau', 'guest_1'] = 'W. Kamau Bell'
data.loc[data.guest_1 == 'B. J.', 'guest_1'] = 'BJ Novak'
data.loc[data.guest_1 == 'Michael K.', 'guest_1'] = 'Michael K. Williams'
data.loc[data.guest_1 == 'Paul F.', 'guest_1'] = 'Paul F. Tompkins'
data.loc[data.guest_1 == 'Thomas L.', 'guest_1'] = 'Thomas L. Friedman'
data.loc[data.guest_1 == 'H. Jon', 'guest_1'] = 'H. Jon Benjamin'
data.loc[data.guest_1 == 'Sterling K.', 'guest_1'] = 'Sterling K. Brown'
data.loc[data.guest_1 == 'Jimmy O.', 'guest_1'] = 'Jimmy O. Yang'
data.loc[data.guest_1 == 'Michael B.', 'guest_1'] = 'Michael B. Jordan'
data.loc[data.guest_1 == 'St. Paul', 'guest_1'] = 'St. Paul and the Broken Bones'
data.loc[data.guest_1 == 'John C.', 'guest_1'] = 'John C. Reilly'
data.loc[data.guest_1 == 'M. Night', 'guest_1'] = 'M. Night Shyamalan'
data.loc[data.guest_1 == 'Taraji P.', 'guest_1'] = 'Taraji P. Henson'
data.loc[data.guest_1 == 'Flight of', 'guest_1'] = 'Flight of the Conchords'
data.loc[data.guest_1 == 'Strand of', 'guest_1'] = 'Strand of Oaks'
data.loc[data.guest_1 == 'Band of', 'guest_1'] = 'Band of Horses'
data.loc[data.guest_1 == 'Florence and', 'guest_1'] = 'Florence and the Machine'
data.loc[data.guest_1 == 'Christine and', 'guest_1'] = 'Christine and the Queens'
data.loc[data.guest_1 == 'Sara &', 'guest_1'] = 'Sara Foster'
data.loc[data.guest_1 == 'Rick &', 'guest_1'] = 'Marty Lagina'
data.loc[data.guest_1 == 'Tim &', 'guest_1'] = 'Eric Wareheim'
data.loc[data.guest_1 == 'Desus &', 'guest_1'] = 'Desus and Mero'
data.loc[data.guest_1 == 'Andrea &', 'guest_1'] = 'Andrea Bocelli'
data.loc[data.guest_1 == 'Bill &', 'guest_1'] = 'Bill Gates'
data.loc[data.guest_1 == 'Mumford &', 'guest_1'] = 'Mumford and Sons'

# PICKLING FILE

In [64]:
with open("../pickles/LSSC_EDA.pkl", 'wb') as LSSC_EDA:
    pickle.dump(data, LSSC_EDA)